In [8]:
import pandas as pd
import os
from tqdm import tqdm

# Nome del file da elaborare
file_name = '/home/jacoponudo/Documents/Size_effects/DATA/facebook/sample_comments.csv'

# Lista per memorizzare i DataFrame elaborati
dataframes = []

# Variabile per contare il totale delle righe
total_rows_count = 0

# Carica solo le colonne richieste dal file CSV
try:
    data = pd.read_csv(file_name, usecols=['created_time', 'post_id', 'from_id'], encoding='ISO-8859-1')
    data.rename(columns={'from_id': 'author_id'}, inplace=True)
    posts = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/facebook/sample_posts.csv', usecols=['page_id', 'post_id'], encoding='ISO-8859-1')
    posts_dict = posts.set_index('post_id').T.to_dict()
    data['page_id'] = data['post_id'].map(lambda x: posts_dict.get(x, {}).get('page_id', None))

except ValueError as e:
    print(f"Missing columns in file {file_name}: {e}")



# Concatenate all DataFrames into a single dataset
final_dataset = data

/tmp/ipykernel_461904/938005116.py:19: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  posts_dict = posts.set_index('post_id').T.to_dict()


In [12]:
final_dataset

,author_id,created_time,post_id,page_id
0,1049996891710740,2015-12-31T16:29:32+0000,157466287640348_923643854355917,157466287640348
1,662218400585314,2015-12-31T21:36:37+0000,157466287640348_923643854355917,157466287640348
2,10153255510262014,2015-12-31T16:28:23+0000,157466287640348_923544961032473,157466287640348
3,10206647671965500,2016-01-01T04:37:38+0000,157466287640348_923544961032473,157466287640348
4,10205508079326181,2016-01-01T06:43:52+0000,157466287640348_923544961032473,157466287640348
...,...,...,...,...
36025478,10153386074810443,2010-12-15T08:52:36+0000,160111214030709_178027758882229,160111214030709
36025479,10208414565656822,2010-12-07T05:27:44+0000,160111214030709_182689331747881,160111214030709
36025480,10207545579490470,2010-12-03T08:04:05+0000,160111214030709_127880867273694,160111214030709
36025481,10208414565656822,2010-11-23T06:18:50+0000,160111214030709_109535849116962,160111214030709


In [14]:
import pandas as pd

# Supponiamo che il tuo DataFrame si chiami `final_data` e contenga le colonne 'post_id' e 'created_at'
# Convertiamo la colonna `created_at` in formato datetime, se non lo è già
final_dataset['created_time'] = pd.to_datetime(final_dataset['created_time'])

# Per ogni `post_id`, troviamo il primo commento e calcoliamo il tempo limite di 100 ore
final_dataset['first_comment_time'] = final_dataset.groupby('post_id')['created_time'].transform('min')
final_dataset['time_diff_hours'] = (final_dataset['created_time'] - final_dataset['first_comment_time']).dt.total_seconds() / 3600

# Filtriamo i commenti che sono arrivati entro le prime 100 ore
filtered_data = final_dataset[final_dataset['time_diff_hours'] <= 100].copy()

# Rimuoviamo le colonne temporanee
filtered_data.drop(['first_comment_time', 'time_diff_hours'], axis=1, inplace=True)

# `filtered_data` ora contiene solo i commenti arrivati nelle prime 100 ore per ciascun `post_id`


In [17]:
data=filtered_data

In [23]:
data['month_year'] = pd.to_datetime(data['created_time'], errors='coerce').dt.to_period('M').astype(str)

# Elimina la colonna 'created_time'
data = data.drop(columns=['created_time'])

,author_id,post_id,page_id,created_at,month_year
0,1049996891710740,157466287640348_923643854355917,157466287640348,2015-12-31 16:29:32+00:00,2015-12
1,662218400585314,157466287640348_923643854355917,157466287640348,2015-12-31 21:36:37+00:00,2015-12
2,10153255510262014,157466287640348_923544961032473,157466287640348,2015-12-31 16:28:23+00:00,2015-12
3,10206647671965500,157466287640348_923544961032473,157466287640348,2016-01-01 04:37:38+00:00,2016-01
4,10205508079326181,157466287640348_923544961032473,157466287640348,2016-01-01 06:43:52+00:00,2016-01
...,...,...,...,...,...
36025478,10153386074810443,160111214030709_178027758882229,160111214030709,2010-12-15 08:52:36+00:00,2010-12
36025479,10208414565656822,160111214030709_182689331747881,160111214030709,2010-12-07 05:27:44+00:00,2010-12
36025480,10207545579490470,160111214030709_127880867273694,160111214030709,2010-12-03 08:04:05+00:00,2010-12
36025481,10208414565656822,160111214030709_109535849116962,160111214030709,2010-11-23 06:18:50+00:00,2010-11


In [25]:


# Incrementa il conteggio totale delle righe
total_rows_count += data.shape[0]

# Step 2: Conta quanti utenti unici ci sono per post
users_per_post = data.groupby('post_id')['author_id'].nunique().reset_index()
users_per_post.columns = ['post_id', 'post_size']

# Step 3: Conta quante volte ogni autore appare per post
author_post_count = data.groupby(['post_id', 'author_id']).size().reset_index(name='interaction_len')

# Aggiungi la colonna 'month_year' al conteggio degli autori
author_post_count = pd.merge(author_post_count, data[['post_id', 'month_year']].drop_duplicates(), on='post_id')

# Step 4: Unisci i due DataFrame su 'post_id'
data = pd.merge(users_per_post, author_post_count, on='post_id')
    


In [26]:
# Salva il dataset finale con la colonna 'month_year' come terza colonna
data[['post_size', 'interaction_len', 'month_years']].to_csv('/home/jacoponudo/Documents/Size_effects/DATA/facebook/PRO_facebook_cutted.csv', index=False)

# Conta il numero di post e di autori unici
num_posts = data['post_id'].nunique()
num_users = data['author_id'].nunique()

print(f"Number of posts: {num_posts}")
print(f"Number of unique users: {num_users}")
print(f"Total number of rows: {total_rows_count}")

Number of posts: 659614
Number of unique users: 6751917
Total number of rows: 105800709
